In [58]:
import json
from collections import OrderedDict
from prompt_toolkit import prompt
from prompt_toolkit.completion import WordCompleter
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from aggiorna_link import aggiorna_link_da_lista
from docx.oxml.ns import qn
import os
import webbrowser
import time

In [93]:
# === CONFIG FILE PATH ===
BASE_DIR = os.getcwd()
LINKS_JSON = os.path.join(BASE_DIR, "video_links.json")
MANCANTI_TXT = os.path.join(BASE_DIR, "mancanti.txt")
OUTPUT_DOC = os.path.join(BASE_DIR, f"scheda prova.docx")
TEMP_JSON = os.path.join(BASE_DIR, "scheda_temp.json")

# === Carica link esercizi da JSON ===
try:
    with open(LINKS_JSON, "r", encoding="utf-8") as f:
        video_links = json.load(f)
except FileNotFoundError:
    video_links = {}


with open(TEMP_JSON, "r", encoding="utf-8") as f:
    scheda_dict = json.load(f)

In [ ]:
esercizi = []
visti = set()
for giorno in scheda_dict.values():
    for esercizio in giorno:
        nome = esercizio["esercizio"].strip().lower()
        if nome not in visti:
            esercizi.append(esercizio["esercizio"].strip().lower())
            visti.add(nome)
esercizi.pop(1)            

In [94]:
video_links = {chiave.strip().lower(): valore for chiave, valore in video_links.items()}

In [99]:
# === Crea documento Word ===
scheda = Document()
# === Sezione video ===
scheda.add_heading("Video esercizi", level=1)
esercizi_stampa = [nome.capitalize() for nome in esercizi]

for nome in esercizi_stampa:
    link = video_links.get(nome.strip().lower())
    testo = f"{nome}: {link}" if link else f"{nome}: 🔗 [manca link]"
    p = scheda.add_paragraph()
    run = p.add_run(testo)
    run.font.name = 'Courier New'
    run._element.rPr.rFonts.set(qn('w:eastAsia'), 'Courier New')

scheda.save("sezione video.docx")